In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/transfusion-dataset/transfusion.data


In [14]:
# Read in dataset
transfusion = pd.read_csv('/kaggle/input/transfusion-dataset/transfusion.data')

# Print out the first rows of our dataset
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [15]:
transfusion.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [16]:
# check the different data types and the number of columns of each
print(transfusion.dtypes.value_counts())

int64    5
dtype: int64


In [17]:
# Print a concise summary of transfusion DataFrame
transfusion.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [18]:
target = transfusion['whether he/she donated blood in March 2007']
transfusion = transfusion.drop(['whether he/she donated blood in March 2007'], axis=1)


In [19]:
transfusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Recency (months)       748 non-null    int64
 1   Frequency (times)      748 non-null    int64
 2   Monetary (c.c. blood)  748 non-null    int64
 3   Time (months)          748 non-null    int64
dtypes: int64(4)
memory usage: 23.5 KB


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(transfusion, target, test_size = 0.2)

In [21]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


model = XGBClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test) 

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.33%


In [29]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=2)
model.fit(X_train, y_train)

predictions = model.predict(X_test) 

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 76.67%


In [44]:
from sklearn.model_selection import GridSearchCV 

n_estimators = [10, 20, 30, 50, 100]
max_depth = [2, 4, 6, 8]
min_samples_split = [2, 5, 10, 15, 30]
min_samples_leaf = [2, 5, 10, 15, 30] 

parameters = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

grid = GridSearchCV(RandomForestClassifier(), parameters, cv = 4, verbose = 1, n_jobs = -1)
best_model = grid.fit(transfusion, target)

Fitting 4 folds for each of 500 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  1.0min finished


In [42]:
print(best_model.best_params_)

{'max_depth': 2, 'min_samples_leaf': 5, 'min_samples_split': 30, 'n_estimators': 50}


In [43]:

print(best_model.best_score_)


0.7794117647058824
